This program will require that you install the following modules in advance: 

If you are on Mac, you may have to download an ffmpeg static build yourself from here, as the pip install and brew install versions are currently broken: https://evermeet.cx/ffmpeg/ (I'll go into more detail on the ffmpeg stuff in the second half of this document) 

If you are running this code on a Mac, run the following block of code after changing the path in line #2 to the path to your ffmpeg build: 

In [13]:
import os #imports the OS module, which we'll use for directory creation
import ffmpeg #essential module for audio processing. it's currently extremely broken on Mac
os.environ["IMAGEIO_FFMPEG_EXE"] = r"./ffmpeg"
import cv2 #essential module for A/V processing. 
from moviepy import * #essential module for A/V processing. 

If you're running this code on a Windows computer, run the following block of code. Note that I only have a Mac computer so I couldn't test this. 

In [14]:
import os #imports the OS module, which we'll use for directory creation
import ffmpeg #essential module for audio processing. 
import cv2 #imports the OpenCV library
from moviepy import * #essential module for A/V processing. 

My goal going into this project is to take the given MP4 and extract the following data:
1. A range of frames from the video, and
2. The video's full audio track

I've used the OpenCV Python library for some computer vision stuff in the past, and I know that it's capable of doing frame extraction, so I'll start there. 

In [15]:
videoPath = "./q2video.mp4"

#chronologically, this next bit of code is the last bit I wrote.
#I got the frames to extract into the same directory as the python
#code, and it became a mess really quickly, so it became obvious
#that I would need to create a data folder to hold all the info we're
#extracting. 

if not os.path.exists('data'):
    os.makedirs('data')

if not os.path.exists('data/frames'):
    os.makedirs('data/frames')

#here's the function to extract a range of frames from a video
def extractFrames(path, firstFrame, lastFrame):

    if lastFrame < firstFrame: 
        print("Frame range can't be negative!")
    
    frameVal = 1 #counter for filenames of frames
    
    video = cv2.VideoCapture(path) #opens video internally
    
    if not video.isOpened(): #checks if above line worked
        print("Failed to open video file") 

    while video.isOpened():
        
        #I couldn't remember the exact function for reading frames, 
        #so I googled the documentation for it, but without even 
        #having to click on anything, Google's new AI overview feature 
        #gave me the exact function I was looking for. I've found this 
        #AI overview feature to be hit-or-miss but in this case it was a hit!
        isOpen, frame = video.read()
    
        if isOpen:
            if frameVal >= firstFrame:
                #writes frame to a file only if it's within the given range at function call
                cv2.imwrite("./data/frames/frame" + str(frameVal) + ".jpg", frame) 

            frameVal += 1
            
            if frameVal - 1 == lastFrame: 
                #closes OpenCV processing and ends function loop when all frames in range are extracted
                video.release()
                cv2.destroyAllWindows()

Everything should be working as intended. The code below will export the first five frames from the video:

In [16]:
extractFrames(videoPath, 1,5)

The code below will export frames ten through twenty.

In [17]:
extractFrames(videoPath, 10, 20)

The code below will return an error. 

In [18]:
extractFrames(videoPath, 20, 10)

Frame range can't be negative!


Next step is to build a function to extract the audio track from the video. OpenCV is a computer vision library and isn't designed to handle audio, so I'll look elsewhere. A brief Google Search tells me that moviepy is a good library to use. 

In [19]:
if not os.path.exists('data/audio'):
    os.makedirs('data/audio')

#I've never used MoviePy before, but it seems really simple to use. 
def extractAudio(videoPath): 
    audioPath = "./data/audio/audio.mp3"
    video = VideoFileClip(videoPath) #loads up video
    audio = video.audio #extracts audio from video

    #audio is starting to no longer look like a real word anymore. do you do any work on semantic satiation?

    audio.write_audiofile(audioPath) #writes audio file

    #close the files
    audio.close()
    video.close()

It took me a whole morning to get ffmpeg working on my Mac. pip install and brew install wouldn't work no matter how hard I tried, so I ended up having to download a static ffmpeg binary. The second line of code above changes the ffmpeg path to point to the static binary I downloaded (I found it on a 4-year-old Reddit post: https://www.reddit.com/r/moviepy/comments/l0177r/ffmpeg_installation_newbie_question/), and after running the code three different times to get my Mac to finally understand that the code is safe to run, I finally got it functional. 

Based on how I wrote this, it should run perfectly on a Windows computer if you just comment out the second line of the above block of code, but I don't have a Windows computer to test this on. 

If you are running the program on a Mac, make sure to change the path to your ffmpeg file at the very top of this document if you haven't already. 

Running the line of code below will extract the audio from the video (It'll spit out some scary warnings, but they don't cause any problems): 

In [20]:
extractAudio(videoPath)

/Users/joelmccullough/my_env/lib/python3.13/site-packages/moviepy/video/io/ffmpeg_reader.py:496: UserWarning: Subtitle stream parsing is not supported by moviepy and will be ignored
ffmpeg output:

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from './q2video.mp4':
  Metadata:
    major_brand     : mp42
    minor_version   : 1
    compatible_brands: isommp41mp42
    creation_time   : 2023-04-11T16:52:04.000000Z
  Duration: 00:01:30.66, start: 0.000000, bitrate: 5635 kb/s
  Stream #0:0[0x1](eng): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, stereo, fltp, 126 kb/s (default)
      Metadata:
        creation_time   : 2023-04-11T16:52:04.000000Z
        handler_name    : Core Media Audio
        vendor_id       : [0][0][0][0]
  Stream #0:1[0x2](eng): Audio: ac3 (ac-3 / 0x332D6361), 48000 Hz, 5.1(side), fltp, 384 kb/s
      Metadata:
        creation_time   : 2023-04-11T16:52:04.000000Z
        handler_name    : Core Media Audio
        vendor_id       : [0][0][0][0]
      Side data:
        audio se

MoviePy - Writing audio in ./data/audio/audio.mp3


MoviePy - Done.


Just for the sake of convenience, I figure I might as well combine these into one single function, just so we can make them run at the same time. 

In [21]:
def extractData(path, firstFrame, lastFrame):
    extractFrames(path, firstFrame, lastFrame)
    extractAudio(path)

And, to test that everything works, this next line should export frames 50 through 70, as well as the audio file: 

In [22]:
extractData(videoPath, 50, 70)

MoviePy - Writing audio in ./data/audio/audio.mp3


MoviePy - Done.
